In [1]:
from bs4 import BeautifulSoup
from random import randint, shuffle
from time import sleep
import re
import pandas as pd
import json
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
# pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.max_info_rows", 200)

In [2]:
df_products = pd.read_csv("final_products.txt")
df_products.head()

,brand_name,product_ref,product_item,product_name,product_price,product_image,product_link,product_price_low,product_price_high
0,Acqua di Parma,P443401,2218774.0,Blu Mediterraneo MINIATURE Set,$66.00,https://www.sephora.com/productimages/sku/s2218774-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/blu-mediterraneo-minature-set-P443401?icid2=products grid:p443401,66.0,66.0
1,Acqua di Parma,P307801,1417609.0,Fico di Amalfi,$30.00 - $180.00,https://www.sephora.com/productimages/sku/s1417609-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/blu-mediterraneo-fico-di-amalfi-P307801?icid2=products grid:p307801,30.0,180.0
2,Acqua di Parma,P163604,2044816.0,Colonia,$66.00 - $170.00,https://www.sephora.com/productimages/sku/s2044816-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/colonia-P163604?icid2=products grid:p163604,66.0,170.0
3,Acqua di Parma,P307804,1417617.0,Mirto di Panarea,$30.00 - $180.00,https://www.sephora.com/productimages/sku/s1417617-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/blu-mediterraneo-mirto-di-panarea-P307804?icid2=products grid:p307804,30.0,180.0
4,Acqua di Parma,P375388,1417567.0,Arancia di Capri,$30.00 - $180.00,https://www.sephora.com/productimages/sku/s1417567-main-zoom.jpg?imwidth=162,https://www.sephora.com/product/blu-mediterraneo-arancia-di-capri-P375388?icid2=products grid:p375388,30.0,180.0


In [159]:
df_products[df_products.product_price_low != df_products.product_price_high].sample(1).product_ref.values[0]

'P423281'

In [3]:
df_products.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5697 entries, 0 to 5696
Data columns (total 9 columns):
brand_name            5697 non-null object
product_ref           5697 non-null object
product_item          3044 non-null float64
product_name          5697 non-null object
product_price         5697 non-null object
product_image         5669 non-null object
product_link          5697 non-null object
product_price_low     5697 non-null float64
product_price_high    5697 non-null float64
dtypes: float64(3), object(6)
memory usage: 400.7+ KB


In [4]:
df_brands = pd.read_csv("brands.txt")
df_brands.head()

,brand_name,brand_link
0,Acqua di Parma,https://www.sephora.com/brand/acqua-di-parma
1,adwoa beauty,https://www.sephora.com/brand/adwoa-beauty
2,AERIN,https://www.sephora.com/brand/aerin-perfume
3,Aether Beauty,https://www.sephora.com/brand/aether-beauty
4,Algenist,https://www.sephora.com/brand/algenist


In [290]:
ref=df_products[df_products.product_price_low != df_products.product_price_high].sample(1).product_ref.values[0]

with open('json/'+ref+'.json') as json_file:
    data = json.load(json_file)

data

FileNotFoundError: [Errno 2] No such file or directory: 'json/P448117.json'

In [261]:
data.keys()

dict_keys(['seoProductName', 'ancillarySkus', 'brand', 'breadcrumbsSeoJsonLd', 'content', 'currentSku', 'displayName', 'enableNoindexMetaTag', 'flashBanner', 'fullSiteProductUrl', 'imageAltText', 'isHideSocial', 'isReverseLookupEnabled', 'longDescription', 'lovesCount', 'navigationSeoJsonLd', 'parentCategory', 'productId', 'productSeoJsonLd', 'productVideos', 'quickLookDescription', 'rating', 'regularChildSkus', 'reviews', 'seoCanonicalUrl', 'seoKeywords', 'seoMetaDescription', 'seoName', 'seoTitle', 'shortDescription', 'skuSelectorType', 'suggestedUsage', 'swatchType', 'targetUrl', 'type', 'ufeSeoCanonicalUrl', 'useItWithTitle', 'variationType', 'variationTypeDisplayName', 'ymalProductTitle', 'ymalSkus', 'biOptions', 'thirdpartyImageHost', 'apiConfigurationData', 'headerFooterTemplate', 'templateInformation'])

In [178]:
k,v= iter(data.keys()), iter(data.values())

In [179]:
next(k),next(v)

('seoProductName', 'miss-dior-blooming-bouquet-P385477')

In [180]:
next(k),next(v)

('ancillarySkus',
 [{'biExclusiveLevel': 'none',
   'brandName': 'Dior',
   'image': '/productimages/sku/s1701630-main-Sgrid.jpg',
   'isAppExclusive': False,
   'isBiOnly': False,
   'isFirstAccess': False,
   'isFree': False,
   'isLimitedEdition': False,
   'isLimitedQuantity': False,
   'isLimitedTimeOffer': False,
   'isNew': False,
   'isOnlineOnly': True,
   'isOnlyFewLeft': False,
   'isOutOfStock': False,
   'isSephoraExclusive': False,
   'listPrice': '$57.00',
   'maxPurchaseQuantity': 10,
   'productId': 'P395905',
   'productName': 'Miss Dior Hair Mist',
   'size': '1 oz/ 30 mL',
   'skuId': '1701630',
   'skuImages': {'image135': '/productimages/sku/s1701630-main-grid.jpg',
    'image162': '/productimages/sku/s1701630-162.jpg',
    'image250': '/productimages/sku/s1701630-main-hero.jpg',
    'image300': '/productimages/sku/s1701630-main-hero-300.jpg',
    'image42': '/productimages/sku/s1701630-main-thumb.jpg',
    'image450': '/productimages/sku/s1701630-main-Lhero.jpg',

In [188]:
data["ancillarySkus"][0]["isLimitedEdition"]
data["ancillarySkus"][0]["size"]
data["ancillarySkus"][0]["listPrice"]
data["ancillarySkus"][0]["skuId"]
data["ancillarySkus"][0]["skuImages"]["image450"]

'/productimages/sku/s1701630-main-Lhero.jpg'

In [189]:
next(k),next(v)

('brand',
 {'brandId': '1073',
  'description': "From catwalk-inspired cosmetics to cutting-edge skin care and timeless fragrances, Christian Dior has brought us some of the most luxurious, trendsetting, and fashionable beauty products around. It's this magical mix of couture-meets-cosmetics that not only attracts celebrities, supermodels, and the society set, but also has solidified Dior's reputation as one of the most unabashedly stylish beauty brands in the world.",
  'displayName': 'Dior',
  'longDescription': "From catwalk-inspired cosmetics to cutting-edge skin care and timeless fragrances, Christian Dior has brought us some of the most luxurious, trendsetting, and fashionable beauty products around. It's this magical mix of couture-meets-cosmetics that not only attracts celebrities, supermodels, and the society set, but also has solidified Dior's reputation as one of the most unabashedly stylish beauty brands in the world.",
  'ref': '900031',
  'targetUrl': '/brand/dior'})

In [190]:
data["brand"]["brandId"]

'1073'

In [191]:
data["brand"]["longDescription"]

"From catwalk-inspired cosmetics to cutting-edge skin care and timeless fragrances, Christian Dior has brought us some of the most luxurious, trendsetting, and fashionable beauty products around. It's this magical mix of couture-meets-cosmetics that not only attracts celebrities, supermodels, and the society set, but also has solidified Dior's reputation as one of the most unabashedly stylish beauty brands in the world."

In [192]:
next(k),next(v)

('breadcrumbsSeoJsonLd',
 '{"itemListElement":[{"item":{"name":"Fragrance","@id":"https://www.sephora.com/shop/fragrance"},"@type":"ListItem","position":1},{"item":{"name":"Women","@id":"https://www.sephora.com/shop/fragrances-for-women"},"@type":"ListItem","position":2},{"item":{"name":"Perfume","@id":"https://www.sephora.com/shop/perfume"},"@type":"ListItem","position":3}],"@type":"BreadcrumbList","@context":"http://schema.org"}')

In [193]:
for i in json.loads(data["breadcrumbsSeoJsonLd"])["itemListElement"]:
    print(i["item"]["name"])

Fragrance
Women
Perfume


In [194]:
next(k),next(v)

('content',
 {'seoCanonicalUrl': '/product/miss-dior-blooming-bouquet-P385477',
  'seoKeywords': [],
  'seoMetaDescription': 'Shop Miss Dior Blooming Bouquet by Dior at Sephora. This vivacious eau de toilette has flirty, floral notes. ',
  'seoName': 'miss-dior-blooming-bouquet-P385477',
  'seoTitle': 'Miss Dior Blooming Bouquet - Dior | Sephora',
  'targetUrl': '/product/miss-dior-blooming-bouquet-P385477'})

In [195]:
data["content"]["seoMetaDescription"]

'Shop Miss Dior Blooming Bouquet by Dior at Sephora. This vivacious eau de toilette has flirty, floral notes. '

In [196]:
next(k),next(v)

('currentSku',
 {'alternateImages': [{'altText': 'Dior Miss Dior Blooming Bouquet Image 2',
    'image135': '/productimages/product/p385477-av-03-grid.jpg',
    'image1500': '/productimages/product/p385477-av-03-zoom.jpg',
    'image162': '/productimages/product/p385477-av-03-162.jpg',
    'image250': '/productimages/product/p385477-av-03-hero.jpg',
    'image300': '/productimages/product/p385477-av-03-hero-300.jpg',
    'image42': '/productimages/product/p385477-av-03-thumb.jpg',
    'image450': '/productimages/product/p385477-av-03-Lhero.jpg',
    'image50': '/productimages/product/p385477-av-03-thumb-50.jpg',
    'image62': '/productimages/product/p385477-av-03-Lthumb.jpg',
    'image97': '/productimages/product/p385477-av-03-Sgrid.jpg'},
   {'altText': 'Dior Miss Dior Blooming Bouquet Image 3',
    'image135': '/productimages/product/p385477-av-04-grid.jpg',
    'image1500': '/productimages/product/p385477-av-04-zoom.jpg',
    'image162': '/productimages/product/p385477-av-04-162.j

In [229]:
data["currentSku"]["alternateImages"][0]["image450"]
# data["currentSku"]["ingredientDesc"]
data["currentSku"]["isLimitedEdition"]
data["currentSku"]["isNaturalOrganic"]
data["currentSku"]["isNaturalSephora"]
data["currentSku"]["listPrice"]
data["currentSku"]["skuName"]
data["currentSku"]["skuId"]
data["currentSku"]["variationValue"]

'1.7 oz/ 50 mL Eau de Toilette Spray'

In [201]:
next(k),next(v)

('displayName', 'Miss Dior Blooming Bouquet')

In [202]:
data["displayName"]

'Miss Dior Blooming Bouquet'

In [203]:
next(k),next(v)

('enableNoindexMetaTag', False)

In [204]:
next(k),next(v)

('flashBanner',
 {'ancestorHierarchy': [{'displayName': 'Unified Ppage Enrollment Flash Text',
    'nodeStatus': 2,
    'targetScreen': {'apiUrl': '/v1/catalog/media/45600023',
     'targetScreen': 'contentstore',
     'targetUrl': '/beauty/unified-ppage-enrollment-flash-text',
     'targetValue': '45600023'}}],
  'mediaType': 68,
  'regions': {'content': [{'componentName': 'Sephora Unified Markdown Component',
     'componentType': 57,
     'contentType': 'PlainText',
     'enableTesting': False,
     'modalComponentTemplate': {'componentName': 'Sephora Unified Modal Component',
      'componentType': 56,
      'components': [{'altText': 'Flash 2-day shipping on all orders | INSIDER VIB ROUGE | ENROLL NOW >',
        'componentName': 'Sephora Unified Image Component',
        'componentType': 53,
        'enableTesting': False,
        'height': '1123',
        'imageId': '6360508',
        'imagePath': '/contentimages/bi_hq/2018_relaunch/benefitschart_modal_images/desktop/2018-bi-rel

In [205]:
next(k),next(v)

('fullSiteProductUrl',
 'https://www.sephora.com/product/miss-dior-blooming-bouquet-P385477?skuId=1599877')

In [206]:
next(k),next(v)

('imageAltText', 'Dior - Miss Dior Blooming Bouquet')

In [207]:
next(k),next(v)

('isHideSocial', False)

In [208]:
next(k),next(v)

('isReverseLookupEnabled', False)

In [209]:
next(k),next(v)

('longDescription',
 "<b>Fragrance Family:</b> Floral<br><br><b>Scent Type:</b> Fruity Florals<br><br><b>Key Notes:</b> Damascus Rose, Peony, White Musk<br><br><b>Fragrance Description:</b> Experience the caress of a fresh rose dressed in peony flowers. This vivacious eau de toilette reveals a tender heart of peony and Damascus rose. The perfume's lingering finish is edged with white musk. A floral springtime signature, perfect for a flirty and irresistibly charming Miss Dior.<br><br><b>About the Bottle:</b> The iconic Miss Dior bottle is inspired by women's couture. The signature Miss Dior bow evokes elegant femininity.<br><br><b>About the Fragrance:</b> Blooming Bouquet is a composition designed like a dress embroidered with a thousand blossoms. It pays tribute to Christian Dior's legendary love of flowers.")

In [210]:
tag_remover = re.compile(r'<[^>]+>')

In [211]:
tag_remover.sub('', data["longDescription"]).split("\r\n\r\n")

["Fragrance Family: FloralScent Type: Fruity FloralsKey Notes: Damascus Rose, Peony, White MuskFragrance Description: Experience the caress of a fresh rose dressed in peony flowers. This vivacious eau de toilette reveals a tender heart of peony and Damascus rose. The perfume's lingering finish is edged with white musk. A floral springtime signature, perfect for a flirty and irresistibly charming Miss Dior.About the Bottle: The iconic Miss Dior bottle is inspired by women's couture. The signature Miss Dior bow evokes elegant femininity.About the Fragrance: Blooming Bouquet is a composition designed like a dress embroidered with a thousand blossoms. It pays tribute to Christian Dior's legendary love of flowers."]

In [212]:
next(k),next(v)

('lovesCount', 47589)

In [213]:
data["lovesCount"]

47589

In [214]:
next(k),next(v)

('navigationSeoJsonLd',
 '{"itemListElement":[{"@type":"siteNavigationElement","name":"Makeup","position":1,"url":"https://www.sephora.com/shop/makeup-cosmetics"},{"@type":"siteNavigationElement","name":"Skincare","position":2,"url":"https://www.sephora.com/shop/skincare"},{"@type":"siteNavigationElement","name":"Hair","position":3,"url":"https://www.sephora.com/shop/hair-products"},{"@type":"siteNavigationElement","name":"Tools & Brushes","position":4,"url":"https://www.sephora.com/shop/makeup-tools"},{"@type":"siteNavigationElement","name":"Fragrance","position":5,"url":"https://www.sephora.com/shop/fragrance"},{"@type":"siteNavigationElement","name":"Bath & Body","position":6,"url":"https://www.sephora.com/shop/bath-body"},{"@type":"siteNavigationElement","name":"Gifts","position":7,"url":"https://www.sephora.com/shop/gifts"},{"@type":"siteNavigationElement","name":"Men","position":8,"url":"https://www.sephora.com/shop/men"},{"@type":"siteNavigationElement","name":"Mini Size","posit

In [215]:
next(k),next(v)

('parentCategory',
 {'categoryId': 'cat60148',
  'displayName': 'Perfume',
  'parentCategory': {'categoryId': 'cat1230039',
   'displayName': 'Women',
   'parentCategory': {'categoryId': 'cat160006',
    'displayName': 'Fragrance',
    'targetUrl': '/shop/fragrance',
    'url': '/catalog/categories/cat160006'},
   'targetUrl': '/shop/fragrances-for-women',
   'url': '/catalog/categories/cat1230039'},
  'targetUrl': '/shop/perfume',
  'url': '/catalog/categories/cat60148/products?sortBy=-1&currentPage=1&content=true'})

In [216]:
data["parentCategory"]["categoryId"]
data["parentCategory"]["displayName"]
data["parentCategory"]["parentCategory"]["categoryId"]
data["parentCategory"]["parentCategory"]["displayName"]
data["parentCategory"]["parentCategory"]["parentCategory"]["categoryId"]
data["parentCategory"]["parentCategory"]["parentCategory"]["displayName"]

'Fragrance'

In [217]:
next(k),next(v)

('productId', 'P385477')

In [218]:
next(k),next(v)

('productSeoJsonLd',
 '{"offers":[{"seller":{"@type":"Organization","name":"Sephora"},"image":"https://www.sephora.com/productimages/sku/s2098044-main-hero.jpg","priceCurrency":"USD","@type":"Offer","price":"38.0","availability":"http://schema.org/InStock","sku":"2098044"},{"seller":{"@type":"Organization","name":"Sephora"},"image":"https://www.sephora.com/productimages/sku/s1725423-main-hero.jpg","priceCurrency":"USD","@type":"Offer","price":"60.0","availability":"http://schema.org/InStock","sku":"1725423"},{"seller":{"@type":"Organization","name":"Sephora"},"image":"https://www.sephora.com/productimages/sku/s1599877-main-hero.jpg","priceCurrency":"USD","@type":"Offer","price":"85.0","availability":"http://schema.org/InStock","sku":"1599877"},{"seller":{"@type":"Organization","name":"Sephora"},"image":"https://www.sephora.com/productimages/sku/s1599869-main-hero.jpg","priceCurrency":"USD","@type":"Offer","price":"107.0","availability":"http://schema.org/InStock","sku":"1599869"},{"sel

In [219]:
next(k),next(v)

('productVideos',
 [{'componentName': 'Sephora Unified Video Component',
   'componentType': 61,
   'enableTesting': False,
   'filePath': '6132067317001',
   'height': '490',
   'name': 'ppage_miss_dior_by_dior_123019_video',
   'overlayFlag': True,
   'startImagePath': '/contentimages/VideoImages/123019/Ppage_MissDiorRosesNRosesEaudeToilette_123019.jpg',
   'styleList': {'FR_CA_HIDE': ''},
   'thumbnailHeight': '150',
   'thumbnailWidth': '150',
   'videoTitle': 'Miss Dior by Dior',
   'videoUrl': '6132067317001',
   'width': '900'},
  {'componentName': 'Sephora Unified Video Component',
   'componentType': 61,
   'enableTesting': False,
   'filePath': '6132069595001',
   'height': '490',
   'name': 'ppage_dior_made_with_love_armelle_122719_video',
   'overlayFlag': True,
   'startImagePath': '/contentimages/VideoImagesNEW/122719/ppage_dior_made_with_love_armelle_122719_video.jpg',
   'styleList': {'FR_CA_HIDE': ''},
   'thumbnailHeight': '150',
   'thumbnailWidth': '150',
   'videoT

In [220]:
next(k),next(v)

('quickLookDescription',
 'A vivacious eau de toilette with flirty, floral notes.')

In [221]:
next(k),next(v)

('rating', 4.3101)

In [222]:
data["rating"]

4.3101

In [223]:
next(k),next(v)

('regularChildSkus',
 [{'alternateImages': [{'altText': 'Dior Miss Dior Blooming Bouquet Image 2',
     'image135': '/productimages/product/p385477-av-03-grid.jpg',
     'image1500': '/productimages/product/p385477-av-03-zoom.jpg',
     'image162': '/productimages/product/p385477-av-03-162.jpg',
     'image250': '/productimages/product/p385477-av-03-hero.jpg',
     'image300': '/productimages/product/p385477-av-03-hero-300.jpg',
     'image42': '/productimages/product/p385477-av-03-thumb.jpg',
     'image450': '/productimages/product/p385477-av-03-Lhero.jpg',
     'image50': '/productimages/product/p385477-av-03-thumb-50.jpg',
     'image62': '/productimages/product/p385477-av-03-Lthumb.jpg',
     'image97': '/productimages/product/p385477-av-03-Sgrid.jpg'},
    {'altText': 'Dior Miss Dior Blooming Bouquet Image 3',
     'image135': '/productimages/product/p385477-av-04-grid.jpg',
     'image1500': '/productimages/product/p385477-av-04-zoom.jpg',
     'image162': '/productimages/produc

In [232]:
type(data["regularChildSkus"])

list

In [228]:
for i in data["regularChildSkus"]:
    print(i["variationValue"],i["size"], i["listPrice"], i["skuId"])

0.7 oz/20 mL Eau de Toilette Rollerball 0.7 oz/20 mL $38.00 2098044
1 oz/ 30 mL Eau de Toilette Spray 1 oz/ 30 mL $60.00 1725423
1.7 oz/ 50 mL Eau de Toilette Spray 1.7 oz/ 50 mL $85.00 1599877
3.4 oz/ 100 mL Eau de Toilette Spray 3.4 oz/ 100 mL $107.00 1599869
5.1oz/150mL Eau de Toilette Spray 5.1oz/150mL $135.00 1770361


In [233]:
next(k),next(v)

('reviews', 1203)

In [234]:
data["reviews"]

1203

In [235]:
next(k),next(v)

('seoCanonicalUrl', '/product/miss-dior-blooming-bouquet-P385477')

In [236]:
next(k),next(v)

('seoKeywords', [])

In [237]:
next(k),next(v)

('seoMetaDescription',
 'Shop Miss Dior Blooming Bouquet by Dior at Sephora. This vivacious eau de toilette has flirty, floral notes. ')

In [238]:
next(k),next(v)

('seoName', 'miss-dior-blooming-bouquet-P385477')

In [239]:
next(k),next(v)

('seoTitle', 'Miss Dior Blooming Bouquet - Dior | Sephora')

In [240]:
next(k),next(v)

('shortDescription',
 "<b>Fragrance Family:</b> Floral<br><br><b>Scent Type:</b> Fruity Florals<br><br><b>Key Notes:</b> Damascus Rose, Peony, White Musk<br><br><b>Fragrance Description:</b> Experience the caress of a fresh rose dressed in peony flowers. This vivacious eau de toilette reveals a tender heart of peony and Damascus rose. The perfume's lingering finish is edged with white musk. A floral springtime signature, perfect for a flirty and irresistibly charming Miss Dior.<br><br><b>About the Bottle:</b> The iconic Miss Dior bottle is inspired by women's couture. The signature Miss Dior bow evokes elegant femininity.<br><br><b>About the Fragrance:</b> Blooming Bouquet is a composition designed like a dress embroidered with a thousand blossoms. ")

In [241]:
next(k),next(v)

('skuSelectorType', 'Image')

In [242]:
next(k),next(v)

('suggestedUsage',
 '<b>Suggested Usage:</b><br>-Fragrance is intensified by the warmth of your own body. Apply in the creases of your knees and elbows for a longer-lasting, stronger scent.\xa0<br>-After applying, avoid rubbing or dabbing skin. This breaks down the fragrance, causing it to wear off more quickly.\xa0<br>-If you prefer placing fragrance on your wrists, be sure to reapply after frequent hand-washing, as this tends to rinse off the scent.\xa0<br>-Replace fragrance after 12 months. Expired perfumes more than a year old lose the integrity of the original scent.<br><br><b> Eau de Toilette or Eau de Parfum?:</b> <br>The difference lies in the volume of perfume oil. While EDT contains five to nine percent, EDP contains more, usually eight to 14 percent. EDPs, therefore, last longer and smell more intense.')

In [243]:
next(k),next(v)

('swatchType', 'Image - 42')

In [244]:
next(k),next(v)

('targetUrl', '/product/miss-dior-blooming-bouquet-P385477')

In [245]:
next(k),next(v)

('type', 'standard')

In [246]:
next(k),next(v)

('ufeSeoCanonicalUrl',
 'https://www.sephora.com/product/miss-dior-blooming-bouquet-P385477')

In [247]:
next(k),next(v)

('useItWithTitle', 'Use It With')

In [248]:
next(k),next(v)

('variationType', 'Size + Concentration + Formulation')

In [249]:
next(k),next(v)

('variationTypeDisplayName', 'Size + Concentration + Formulation')

In [250]:
next(k),next(v)

('ymalProductTitle', 'Other Scents You Might Like')

In [251]:
next(k),next(v)

('ymalSkus',
 [{'biExclusiveLevel': 'none',
   'brandName': 'Dior',
   'image': '/productimages/sku/s857888-main-hero.jpg',
   'isAppExclusive': False,
   'isBiOnly': False,
   'isFirstAccess': False,
   'isFree': False,
   'isLimitedEdition': False,
   'isLimitedQuantity': False,
   'isLimitedTimeOffer': False,
   'isNew': False,
   'isOnlineOnly': False,
   'isOnlyFewLeft': False,
   'isOutOfStock': False,
   'isSephoraExclusive': False,
   'listPrice': '$85.00',
   'maxPurchaseQuantity': 10,
   'productId': 'P375238',
   'productName': 'Dior Addict Eau Fraîche',
   'skuId': '857888',
   'skuImages': {'image135': '/productimages/sku/s857888-main-grid.jpg',
    'image1500': '/productimages/sku/s857888-main-zoom.jpg',
    'image162': '/productimages/sku/s857888-162.jpg',
    'image250': '/productimages/sku/s857888-main-hero.jpg',
    'image300': '/productimages/sku/s857888-main-hero-300.jpg',
    'image42': '/productimages/sku/s857888-main-thumb.jpg',
    'image450': '/productimages/sk

In [252]:
next(k),next(v)

('biOptions',
 {'ageRange': [{'displayName': '13-17', 'value': '13-17'},
   {'displayName': '18-24', 'value': '18-24'},
   {'displayName': '25-34', 'value': '25-34'},
   {'displayName': '35-44', 'value': '35-44'},
   {'displayName': '45-54', 'value': '45-54'},
   {'displayName': 'Over 54', 'value': 'over 54'}],
  'eyeColor': [{'displayName': 'Blue', 'value': 'blue'},
   {'displayName': 'Brown', 'value': 'brown'},
   {'displayName': 'Green', 'value': 'green'},
   {'displayName': 'Grey', 'value': 'grey'},
   {'displayName': 'Hazel', 'value': 'hazel'}],
  'gender': [{'displayName': 'Male', 'value': 'male'},
   {'displayName': 'Female', 'value': 'female'}],
  'hairColor': [{'displayName': 'Blonde', 'value': 'blonde'},
   {'displayName': 'Brunette', 'value': 'brunette'},
   {'displayName': 'Auburn', 'value': 'auburn'},
   {'displayName': 'Black', 'value': 'black'},
   {'displayName': 'Red', 'value': 'red'},
   {'displayName': 'Gray', 'value': 'gray'}],
  'hairConcerns': [{'displayName': 'An

In [253]:
next(k),next(v)

('thirdpartyImageHost', 'https://api.sephora.com')

In [254]:
next(k),next(v)

('apiConfigurationData',
 {'CCPABannerEnabled': True,
  'appManagedURLs': ['www.sephora.com',
   'skedge.me',
   'sephora.skedge.me',
   'qa-sephora.skedge.me',
   '*.google.com',
   'survey.medallia.com',
   '*.timetradesystems.com',
   'timetrade.com',
   '*.sephora.com'],
  'backInStockReminderForStoreMsgEnabled': False,
  'basketReminderForStoreMsgEnabled': False,
  'bookReservationUrl': 'https://timetrade.com/app/sephora/workflows/SEPH001/schedule?ch=MOBILEWEB',
  'bopisConfigurations': {'showSameday': False,
   'showSamedayBasket': False,
   'showSamedayFor0202': False,
   'showSamedayFor0336': False,
   'showSamedayFor0356': False,
   'showSamedayFor0398': False,
   'showSamedayFor0460': False,
   'showSamedayFor0922': False,
   'showSamedayHomepage': False,
   'showSamedayNlevelpage': False,
   'showSamedayOrderhistorypage': False,
   'showSamedayPpage': False},
  'branchNetworkTimeout': '5500',
  'brightEdgeConfigurations': {'accountId': 'f00000000049176',
   'dataId': 'bec',


In [255]:
next(k),next(v)

('headerFooterTemplate',
 {'ancestorHierarchy': [{'displayName': 'Header And Footer Template',
    'nodeStatus': 2}],
  'mediaType': 67,
  'navigationSeoJsonLd': '{"itemListElement":[{"@type":"siteNavigationElement","name":"Makeup","position":1,"url":"https://www.sephora.com/shop/makeup-cosmetics"},{"@type":"siteNavigationElement","name":"Skincare","position":2,"url":"https://www.sephora.com/shop/skincare"},{"@type":"siteNavigationElement","name":"Hair","position":3,"url":"https://www.sephora.com/shop/hair-products"},{"@type":"siteNavigationElement","name":"Tools & Brushes","position":4,"url":"https://www.sephora.com/shop/makeup-tools"},{"@type":"siteNavigationElement","name":"Fragrance","position":5,"url":"https://www.sephora.com/shop/fragrance"},{"@type":"siteNavigationElement","name":"Bath & Body","position":6,"url":"https://www.sephora.com/shop/bath-body"},{"@type":"siteNavigationElement","name":"Gifts","position":7,"url":"https://www.sephora.com/shop/gifts"},{"@type":"siteNavigati

In [256]:
next(k),next(v)

('templateInformation', {'template': 'Product/ProductPage', 'channel': 'FS'})